In [ ]:
# pip install selenium

In [12]:
# pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


파일 이름, 디렉터리 이름 또는 볼륨 레이블 구문이 잘못되었습니다.


In [ ]:
# pip install xlsxwriter
#     result.to_excel("beer_n_"+str(k)+".xlsx", engine='xlsxwriter')
# engine='xlsxwriter' 추가

In [16]:
# 사진을 저장하기 위해서 몇가지가 추가 되었다
import pandas as pd
import numpy as np
import time

import re


import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait           #추가
from selenium.common.exceptions import NoSuchElementException     #추가  
import urllib                                                     #추가
import urllib.request                                             #추가

# beer_list를 만들기위한 작업

In [77]:
# 사용할 데이터 : 맥주이름.csv
beer_list = pd.read_csv('맥주이름.csv', encoding='utf-8')

In [78]:
# list확인
beer_list

,Unnamed: 0,맥주이름
0,0,Kloud Original Gravity
1,1,Kloud Fitz Super Clear
2,2,Asahi Super Dry
3,3,Tsingtao
4,4,Heineken
...,...,...
70,70,Red Rock
71,71,Erdinger Weissbier Dunkel
72,72,Warsteiner Premium Verum
73,73,Queen's Ale Blonde Type


In [79]:
# 컬럼 확인
beer_list.columns
#'Unnamed: 0'이라는 필요 없는 컬럼이 있다

Index(['Unnamed: 0', '맥주이름'], dtype='object')

In [80]:
# column정리
# ['Unnamed: 0', '맥주이름'] 두개 컬럼이 있는데 '맥주이름'컬럼만 사용 할 것이다.
beer_list = beer_list[['맥주이름']]

In [81]:
# Hoegaarden은 xpath위치가 다르기 때문에 제외해준다
# 마지막에 호가든은 추기 되기 때문에 사이트에 접속해서 jpg파일을 직접 저장하겠다. 

# 그 외 특이사항으로 이름이 검색되지 않아 확인해본 결과 다른 이름으로 검색하면 검색이 가능하다
# beer_n_47.xlsx(beer_n_47.csv)'Bali Hai Premium Larger' = 'Bali Hai' 로 수정

beer_list.drop([74], axis=0, inplace=True)

C:\Users\adele\anaconda3\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [82]:
# 수집할 맥주 목록을 list로 변환
beer_list = beer_list['맥주이름'].tolist()

In [83]:
# list로 변환된 맥주 이름
beer_list

['Kloud Original Gravity',
 'Kloud Fitz Super Clear',
 'Asahi Super Dry',
 'Tsingtao',
 'Heineken',
 'Kirin Ichiban',
 'Sapporo Premium Beer / Draft Beer',
 'Stella Artois',
 'Guinness Draught',
 'Kronenbourg 1664 Blanc',
 'Pilsner Urquell',
 'San Miguel',
 'OB Premier Pilsner',
 'Cass Fresh',
 'Hite D (Dry Finish)',
 'Hite Prime Max',
 'Hite Extra Cold',
 'Victoria Bitter',
 'Bintang Pilsener',
 'Krombacher Weizen',
 'Miller Genuine Draft (MGD)',
 'Hoegaarden Cherry',
 'Tiger Radler Lemon',
 "Suntory The Premium Malt's",
 'Reeper B. Blondes Weissbier',
 'Bright Leaf Peeper Red IPA',
 'Tiger Beer',
 'Tsingtao Wheat Beer',
 'Erdinger Weissbier',
 'Carlsberg Pilsner',
 'Budweiser',
 'Sapporo Yebisu',
 'Paulaner Hefe-Weissbier',
 'Desperados',
 'Peroni Nastro Azzurro',
 'Edelweiss Weissbier Snowfresh',
 'Heineken Dark Lager',
 'Kozel Černý (Dark) 10°',
 'Guinness Original 4.2% (Ireland/UK)',
 'FiLite',
 'ARK Seoulite Ale',
 'Jeju Wit Ale',
 'Stephans Bräu Premium Pilsener',
 'Stephans Brä

In [99]:
len(beer_list)

74

In [ ]:
# 이미지 xpath : //*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[1]/div/div[2]/div[1]/img

# selenium, xpath를 이용한 이미지저장 코드
# img_num = 1
#   while True:
#       try:
#           img = driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div/div[1]/div[' + str(img_num) + ']/img')
#           src = img.get_attribute('src')
#           urllib.request.urlretrieve(src,'이미지 이름_%03d.png'%(img_num))
#       except NoSuchElementException:  #마지막 이미지 저장 후 반복문을 빠져나감.
#           break
#       img_num = img_num+1

In [85]:
# 데이터프레임으로 저장
beer_list = pd.DataFrame(data=beer_list, columns=['검색이름'])

In [86]:
# 데이터프레임 확인
beer_list

,검색이름
0,Kloud Original Gravity
1,Kloud Fitz Super Clear
2,Asahi Super Dry
3,Tsingtao
4,Heineken
...,...
69,Berliner Kindl Jubiläums Pilsener Premium
70,Red Rock
71,Erdinger Weissbier Dunkel
72,Warsteiner Premium Verum


In [100]:
# 데이터 프레임 생성
data = pd.DataFrame(data=[], columns=['맥주정보', '검색이름', '맥주이름'])

# chromedriver.exe 파일 경로 설정
chromedriver = 'C:/Users/adele/sel/chromedriver.exe'
# 크롤링 할 경로 설정
url = 'https://www.ratebeer.com/search?tab=beer'

# 셀레니움으로 웹브라우저를 오픈합니다.
driver = webdriver.Chrome(chromedriver)
driver.get(url)
driver.set_window_size(900, 900)
time.sleep(1)

In [96]:
def crawl(driver, beer, data, k):
    # 데이터 프레임 생성
    data = pd.DataFrame(data=[], columns=['맥주정보', '검색이름', '맥주이름'])

    # url open
    print('url_open... {0} 맥주 사진을 수집합니다..'.format(beer))
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    driver.set_window_size(900, 900)

    # 1번 사진에 해당 : 맥주 검색
    time.sleep(2)
    element = driver.find_element_by_xpath('//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div/input')
    time.sleep(2)
    element.click()
    time.sleep(2)
    element.send_keys(beer)
    time.sleep(3)
    
    # 2번 사진에 해당 : 상품 선택
    driver.find_element_by_xpath('//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[1]/div/div[2]').click()
    # 호가든 xpath : //*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[2]/div 

    # 3번 사진에 해당 : 상품 이름 수집
    time.sleep(3)
    beer_name = driver.find_element_by_css_selector('.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4').text

    error_cnt = 0
    
    # 이미지 xpath : //*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[1]/div/div[2]/div[1]/img
    img_num = 1
    while 1:
        try:
          img = driver.find_element_by_xpath('//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[1]/div/div[2]/div[' + str(img_num) + ']/img')
          src = img.get_attribute('src')
          fi = urllib.request.urlretrieve(src, 'beer_'+str(k)+'.jpg')#(beer_+'str(k)'+'.jpg')
          img_num=img_num+1
        except NoSuchElementException:  #마지막 이미지 저장 후 반복문을 빠져나감.
          break
        img_num = img_num+1

    driver.quit()

    return 

In [97]:
# 사진저장 자동화 함수 실행 코드
for k in range(len(beer_list)):
    result = crawl(driver, beer_list['검색이름'].iloc[k], data, k)

url_open... Kloud Original Gravity 맥주 데이터를 수집합니다..
url_open... Kloud Fitz Super Clear 맥주 데이터를 수집합니다..
url_open... Asahi Super Dry 맥주 데이터를 수집합니다..
url_open... Tsingtao 맥주 데이터를 수집합니다..
url_open... Heineken 맥주 데이터를 수집합니다..
url_open... Kirin Ichiban 맥주 데이터를 수집합니다..
url_open... Sapporo Premium Beer / Draft Beer 맥주 데이터를 수집합니다..
url_open... Stella Artois 맥주 데이터를 수집합니다..
url_open... Guinness Draught 맥주 데이터를 수집합니다..
url_open... Kronenbourg 1664 Blanc 맥주 데이터를 수집합니다..
url_open... Pilsner Urquell 맥주 데이터를 수집합니다..
url_open... San Miguel 맥주 데이터를 수집합니다..
url_open... OB Premier Pilsner 맥주 데이터를 수집합니다..
url_open... Cass Fresh 맥주 데이터를 수집합니다..
url_open... Hite D (Dry Finish) 맥주 데이터를 수집합니다..
url_open... Hite Prime Max 맥주 데이터를 수집합니다..
url_open... Hite Extra Cold 맥주 데이터를 수집합니다..
url_open... Victoria Bitter 맥주 데이터를 수집합니다..
url_open... Bintang Pilsener 맥주 데이터를 수집합니다..
url_open... Krombacher Weizen 맥주 데이터를 수집합니다..
url_open... Miller Genuine Draft (MGD) 맥주 데이터를 수집합니다..
url_open... Hoegaarden Cherry 맥주 데이터를 수집합니다.